In [ ]:
from keras.datasets import mnist
from keras.layers import *
from keras.layers.advanced_activations import LeakyReLU
from keras.models import Sequential,Model
from keras.optimizers import adam
import numpy as np
import math
import matplotlib.pyplot as plt

In [ ]:
(x_train,_),(_,_)=mnist.load_data()

In [ ]:
print(x_train.shape) 
x_train=x_train.reshape((*x_train.shape,1))

In [ ]:
plt.imshow(x_train[0],cmap='gray')

In [ ]:
x_train=(x_train.astype('float32')-127.5)/127.5 
print(np.min(x_train))
print(np.max(x_train))

In [ ]:
total_epochs=50
batch_size=256
no_of_batches=int(x_train.shape[0]/batch_size)
half_batch=int(batch_size/2)
noise_dim=100
adam=adam(lr=2e-4,beta_1=0.5)

In [ ]:
generator=Sequential()
generator.add(Dense(256,input_shape=(noise_dim,))) 
generator.add(LeakyReLU(0.2))
generator.add(Dense(512)) 
generator.add(LeakyReLU(0.2))
generator.add(Dense(1024))
generator.add(LeakyReLU(0.2))
generator.add(Dense(784,activation='tanh'))
generator.compile(loss='binary_crossentropy',optimizer='adam')

In [ ]:
discriminator=Sequential()
discriminator.add(Dense(512,input_shape=(784,)))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dense(256))
discriminator.add(LeakyReLU(0.2))
discriminator.add(Dense(1,activation='sigmoid'))
discriminator.compile(loss='binary_crossentropy',optimizer='adam')

In [ ]:
discriminator.trainable=False
gan_input=Input(shape=(noise_dim,)) 
generated_img=generator(gan_input) 
gan_output=discriminator(generated_img)

model=Model(gan_input,gan_output) 


model.compile(loss='binary_crossentropy',optimizer='adam')

In [ ]:
x_train=x_train.reshape(-1,784)

In [ ]:
def save_images(epoch,samples):
  noise=np.random.normal(0,1,size=(samples,noise_dim))
  generated_imgs=generator.predict(noise)
  generated_imgs=generated_imgs.reshape(samples,28,28)
  plt.figure(figsize=(10,10))
  for i in range(samples):
    plt.subplot(10,10,i+1) 
    plt.imshow(generated_imgs[i],interpolation='nearest',cmap='gray')
    plt.axis("off")
  plt.tight_layout()
  plt.savefig('images/gan_output_epoch_{0}.png'.format(epoch+1))
  plt.show()

In [ ]:
!mkdir images
!mkdir models
!ls

In [ ]:
d_losses=[]
g_losses=[]
for e in range(total_epochs):
  epoch_d_loss=0.0 
  epoch_g_loss=0.0 
  for step in range(no_of_batches):
    idx=np.random.randint(0,x_train.shape[0],half_batch)
    real_imgs=x_train[idx]
    noise=np.random.normal(0,1,size=(half_batch,noise_dim))  
    fake_imgs=generator.predict(noise)
    real_y=np.ones((half_batch,1))*0.9
    fake_y=np.zeros((half_batch,1))
    d_loss_real=discriminator.train_on_batch(real_imgs,real_y)
    d_loss_fake=discriminator.train_on_batch(fake_imgs,fake_y)
    d_loss=(0.5*d_loss_real)+(0.5*d_loss_fake)
    epoch_d_loss+=d_loss
    noise=np.random.normal(0,1,size=(batch_size,noise_dim))
    ground_truth_y=np.ones((batch_size,1))
    g_loss=model.train_on_batch(noise,ground_truth_y)
    epoch_g_loss=g_loss    
  print("Epoch %d\nDiscriminator loss %.4f\nGenerator Loss %.4f"%(e+1,epoch_d_loss/no_of_batches,epoch_d_loss/no_of_batches))
  d_losses.append(epoch_d_loss/no_of_batches)
  g_losses.append(epoch_g_loss/no_of_batches)
  if (e+1)%5==0:
    generator.save('models/gan_generator_{0}.h5'.format(e+1)) 
    save_images(e,100)

In [ ]:
plt.plot(d_losses,label="Discriminator")
plt.plot(g_losses,label="Generator")
plt.legend()
plt.show()

In [ ]:
!zip -r /content/images.zip /content/images

In [ ]:
from google.colab import files
files.download('images.zip')